the model consists of 3 layers:
* first: input layer spec
* second: "hidden" neuron layer. 128 neurons, random init, relu will allow flowing to next level if a value is > 0
* third: output layer. 10 neurons per 10 possible outputs (classes)

## train to a specific accuracy using callbacks

In [42]:
import tensorflow as tf

class Trainer(tf.keras.callbacks.Callback):
    def __init__(self, desired_accuracy):
        self.desired_accuracy = desired_accuracy
        
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') >= self.desired_accuracy):
            print(f'\nreached desired accuracy: {self.desired_accuracy}, stopping training.')
            self.model.stop_training = True


mnist = tf.keras.datasets.fashion_mnist

(training_images, training_labels), (test_images, test_labels) = mnist.load_data()

# normalize:
training_images = training_images / 255.0
test_images = test_images / 255.0

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

trainer = Trainer(0.95)

model.fit(training_images, training_labels, epochs=50, callbacks=[trainer])

Epoch 1/50
1875/1875 [==============================] - 1s 597us/step - loss: 0.4998 - accuracy: 0.8250
Epoch 2/50
1875/1875 [==============================] - 1s 601us/step - loss: 0.3741 - accuracy: 0.8649
Epoch 3/50
1875/1875 [==============================] - 1s 574us/step - loss: 0.3365 - accuracy: 0.8770
Epoch 4/50
1875/1875 [==============================] - 1s 578us/step - loss: 0.3132 - accuracy: 0.8850
Epoch 5/50
1875/1875 [==============================] - 1s 580us/step - loss: 0.2950 - accuracy: 0.8912
Epoch 6/50
1875/1875 [==============================] - 1s 576us/step - loss: 0.2804 - accuracy: 0.8975
Epoch 7/50
1875/1875 [==============================] - 1s 572us/step - loss: 0.2680 - accuracy: 0.8998
Epoch 8/50
1875/1875 [==============================] - 1s 566us/step - loss: 0.2581 - accuracy: 0.9034
Epoch 9/50
1875/1875 [==============================] - 1s 580us/step - loss: 0.2471 - accuracy: 0.9086
Epoch 10/50
1875/1875 [==============================] - 1s 589u

In [43]:
model.evaluate(test_images, test_labels)

313/313 [==============================] - 0s 401us/step - loss: 0.4191 - accuracy: 0.8923


[0.41906341910362244, 0.892300009727478]


## how to determine success rate found in model.evaulate()
* the `test_labels` array contains the expected classification (0 - 9)
* the `classifications` array contains 9 values, highest index is the result

In [45]:
classifications = model.predict(test_images)
total = 0
failed = 0
for i in range(10000):
    total += 1
    expect = test_labels[i]
    actual = classifications[i].argmax(axis=0)
    try:
        assert expect == actual
    except:
        failed += 1
success = total - failed
rate = round((success / total) * 100.0)
print(f'{rate}% success rate. found {failed} classification failures out of {total} tests')

313/313 [==============================] - 0s 379us/step
89% success rate. found 1077 classification failures out of 10000 tests
